In [ ]:
import sys
sys.path.append("../")
from mir.common import SegmentLibrary
from mir.distances import aligner

In [ ]:
lib = SegmentLibrary.load_default(genes = ['TRA', 'TRB'])
print(lib.get_summary())
print(lib.get_genes())

In [ ]:
seqs = lib.get_seqaas('TRB', 'V')
print(seqs[0:3])

In [ ]:
galn = aligner.AlignGermline.from_seqs(seqs)
print(galn.dist)
print(galn.score_norm('TRBV10-1*01', 'TRBV10-1*01'))
print(galn.score_norm('TRBV10-1*01', 'TRBV10-2*01'))

In [ ]:
import pandas as pd
df = pd.DataFrame([(g1, g2, s, galn.score_norm(g1, g2)) for ((g1, g2), s) in galn.dist.items()], 
                  columns = ['Segm1', 'Segm2', 'Score', 'Score_norm'])
print(df)

In [ ]:
import re
import seaborn as sns
df_sfam = pd.DataFrame([(g, re.split('[-\\*]', g)[0]) for (g, s) in seqs],
                       columns = ['Segm', 'Fam'])

codes, uniques = pd.factorize(df_sfam['Fam'])
palette = zip(uniques, sns.color_palette(None, len(uniques)).as_hex())
df_sfam = df_sfam.merge(pd.DataFrame(palette, columns = ['Fam', 'Col']))
cols = dict(df_sfam[['Segm', 'Col']].itertuples(index=False))

In [ ]:
df1 = df.pivot(index = "Segm1", columns = "Segm2", values = "Score")
sns.clustermap(df1, cmap="mako", 
               row_colors=df1.index.map(cols),
               col_colors=df1.index.map(cols))

In [ ]:
from numpy import exp
df2 = df.pivot(index = "Segm1", columns = "Segm2", values = "Score_norm")
sns.clustermap(exp(df2/300), cmap="mako", 
               row_colors=df2.index.map(cols),
               col_colors=df2.index.map(cols), 
               xticklabels=True,
               yticklabels=True)